In [1]:
import io
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

def response_to_df(response):
    data = io.StringIO(response)
    try:
        return pd.read_csv(data)
    except:
        raise RuntimeError(f"Error while parsing response:\n{response}")

def use_model(model, template, **params):
    prompt = ChatPromptTemplate.from_template(template)
    chain = (
            prompt
            | model
            | StrOutputParser()
    )
    return response_to_df(chain.invoke(params))

template = """
Finde im folgenden Webseiten-Text die Herausgeber, Redaktion/Schriftleitung und Beirat der Zeitschrift '{journal_name}' und gebe sie im CSV-Format zurück mit den Spalten 'lastname', 'firstname', 'title', 'position', 'affiliation','role'. Die Spalte 'role' enthält entweder 'Herausgeber', 'Redaktion', 'Beirat', 'Schriftleitung' oder ist leer wenn nicht bestimmbar.Wenn die Zeitschrift von einer juristischen Person (Verein, Universität, Institut etc.) herausgegeben wird, gebe den Namen der Institution in der Spalte 'lastname' an. Wenn keine passenden Informationen verfügbar sind, gebe nur den CSV-Header zurück. Setze alle Werte in den CSV-Spalten in Anführungszeichen. Antworte ohne zusätzlichen Kommentar, sondern nur mit den CSV-Daten.

{website_text}
"""

with open('data/input/journal-website.txt', encoding='utf-8') as f:
    website_text = f.read()
journal_name = "AUR - Agrar- und Umweltrecht"

## ChatGPT-4 

see [result](data/output/editors-openai-gpt-4.csv)

In [14]:
model = ChatOpenAI(model_name="gpt-4")
df = use_model(model, template, journal_name=journal_name, website_text=website_text)
df.to_csv('data/output/editors-openai-gpt-4.csv')
df

,lastname,firstname,title,position,affiliation,role
0,Deutsche Gesellschaft für Agrarrecht,NaN,NaN,NaN,NaN,Herausgeber
1,Busse,Christian,Dr.,Regierungsdirektor,Bundesministerium für Ernährung und Landwirtsc...,Redaktion
2,Endres,Ewald,Prof. Dr.,NaN,"Hochschule Weihenstephan-Triesdorf, Freising",Redaktion
3,Francois,Matthias,Dr.,Rechtsanwalt,NaN,Redaktion
4,von Garmissen,Bernd,Dr.,Rechtsanwalt,NaN,Redaktion
5,Glas,Ingo,Dr.,Rechtsanwalt,NaN,Redaktion
6,Graß,Christiane,NaN,Rechtsanwältin,NaN,Redaktion
7,Haarstrich,Jens,NaN,Rechtsanwalt,NaN,Redaktion
8,Koch,Erich,Dr.,Ltd. Verwaltungsdirektor,"Sozialversicherung für Landwirtschaft, Forsten...",Redaktion
9,Köpl,Christian,Dr.,Ministerialrat,"Bayerisches Staatsministerium für Ernährung, L...",Redaktion


## ChatGPT 3.5-turbo

see [result](data/output/editors-openai-gpt-3.5-turbo.csv)


In [15]:
model = ChatOpenAI(model_name="gpt-3.5-turbo")
df = use_model(model, template, journal_name=journal_name, website_text=website_text)
df.to_csv('data/output/editors-openai-gpt-3.5-turbo.csv')
df

,lastname,firstname,title,position,affiliation,role
0,Deutsche Gesellschaft für Agrarrecht,,,Herausgeber,,
1,Regierungsdirektor,Dr. Christian,,Redaktion,Bundesministerium für Ernährung und Landwirtsc...,
2,Prof. Dr.,Ewald,,Redaktion,"Hochschule Weihenstephan-Triesdorf, Freising",
3,Rechtsanwalt,Dr. Matthias,,Redaktion,Bitburg,
4,Rechtsanwalt,Dr. Bernd,von Garmissen,Redaktion,Göttingen,
5,Rechtsanwalt,Ingo,,Redaktion,Rostock,
6,Rechtsanwältin,Christiane,,Redaktion,Bonn,
7,Rechtsanwalt,Jens,,Redaktion,Peine,
8,Ltd. Verwaltungsdirektor,Dr. Erich,,Redaktion,"Sozialversicherung für Landwirtschaft, Forsten...",
9,Ministerialrat,Dr. Christian,,Redaktion,"Bayerisches Staatsministerium für Ernährung, L...",


## TheBloke/Llama-2-13B-chat-GPTQ via Huggingface Inference Endpoint

https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ

In [2]:
from lib.hf_llama2_13b_chat import query

lines = query(template, journal_name=journal_name, website_text=website_text).split("\n")[4:]
lines

['',
 'Martinez,José,Professor',
 'Busse,Regierungsdirektor',
 'Ewald,Endres,Forst,Jurist',
 'Glas,VolkmarNies,Koch',
 '',
 '',
 'Note: Some positions have been omitted as they are not explicitly mentioned in the text.']